In [10]:
import json
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math

In [13]:
od_data = np.load('od_matrix.npy')
duration_data = np.load('duration_matrix.npy')
distance_data = np.load('distance_matrix.npy')
fleet_size = 116616*0.088*2.5 # got number from Justin: 116616
number_charging_stations = fleet_size*0.05

od_data = od_data[:, :, 12:-4] # in requests
duration_data = duration_data[:, :, 12:-4] # in sec
distance_data = distance_data[:, :, 12:-4] # in miles
demand_per_time = np.zeros(od_data.shape[2])
for t in range(od_data.shape[2]):
    demand_per_time[t] = od_data[:,:,t].sum()
print("max demand", demand_per_time.max())
low_demand_times = demand_per_time < demand_per_time.max()*0.8
print("low_demand_times", low_demand_times)
peak_hours = []
for time in range(len(low_demand_times)):
    if not low_demand_times[time]:
        peak_hours.append(time)
total_demand = od_data.sum()
print("total_demand", total_demand)
demand_distr = np.zeros(5)
charging_stations = np.zeros(5)
for region in range(5):
    demand_distr[region] = od_data[region,:,:].sum()/total_demand
    charging_stations[region] = math.ceil(demand_distr[region]*number_charging_stations)

delta_c = 2. # energy step [kWh] 0.75
time_granularity = 0.5 # in h

chevy_bolt_capacity = 65 # in kWh
chevy_bolt_usable_capacity = chevy_bolt_capacity * 0.6 # never go below 20% or above 80% of charge
charger_capacity = 50.0 # assuming 50. kW Chargers
assert (delta_c/charger_capacity)/time_granularity < 1
charge_levels_per_charge_step = math.floor((charger_capacity*time_granularity)/delta_c)
chevy_bolt_range = 230 # range in mi for mild city trips according to https://media.chevrolet.com/media/us/en/chevrolet/2022-bolt-euv-bolt-ev.detail.html/content/Pages/news/us/en/2021/feb/0214-boltev-bolteuv-specifications.html
chevy_bolt_usable_range = chevy_bolt_range*0.6 # never go below 20% or above 80% of charge and assume 10% less efficient because of range https://cleantechnica.com/2017/10/13/autonomous-cars-shorter-range-due-high-power-consumption-computers/
chevy_bolt_kwh_per_mi = (chevy_bolt_usable_capacity/chevy_bolt_usable_range)/0.7 # we lose 30% efficiency because of AVs (https://medium.com/@teraki/energy-consumption-required-by-edge-computing-reduces-a-autonomous-cars-mileage-with-up-to-30-46b6764ea1b7)
energy_distance = np.ceil(((distance_data * chevy_bolt_kwh_per_mi)/delta_c).max(axis=2))
energy_distance[energy_distance==0] = 1 # we should always use energy to satisfy a trip
print(energy_distance)
np.save('energy_distance.npy', energy_distance)
duration_data = np.round(duration_data/(3600*time_granularity)) # convert travel time from sec to h
duration_data[duration_data==0] = 1. # it should always take time to satisfy a trip
print("mean time", duration_data.mean())
data_timespan = od_data.shape[2]
episode_length = int(data_timespan/time_granularity)
# https://www.pge.com/tariffs/assets/pdf/tariffbook/ELEC_SCHEDS_BEV.pdf for prices of energy
p_energy = np.ones(episode_length)*0.16872 # in $/kWh
p_energy[int(16/time_granularity):int(21/time_granularity)] = 0.38195 # peak prices
p_energy[int(9/time_granularity):int(14/time_granularity)] = 0.14545 # super off peak prices
p_energy *= delta_c # in $/ charge level
p_travel = 0.0770 # [$ / mi] https://newsroom.aaa.com/wp-content/uploads/2021/08/2021-YDC-Brochure-Live.pdf 
avg_miles_per_h_driving = 9.6 # in m/h from 190x190x24
operational_cost_per_timestep = avg_miles_per_h_driving * time_granularity * p_travel

peak_demand_additional_vehicles = 0

number_chargelevels = int(chevy_bolt_usable_capacity/delta_c)
number_spatial_nodes = 5
charge_locations = np.ones(number_spatial_nodes,dtype=bool).tolist()
cars_per_station_capacity = charging_stations.tolist()

print(charge_levels_per_charge_step)
print(number_chargelevels)
print(fleet_size)

max demand 33269.63333333334
low_demand_times [ True  True False False False False False False]
total_demand 232067.09999999998
[[1. 2. 2. 3. 5.]
 [2. 1. 4. 2. 3.]
 [2. 4. 1. 3. 7.]
 [3. 2. 3. 1. 6.]
 [5. 3. 7. 6. 1.]]
mean time 1.24
12
19
25142.4096


In [14]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
temp_demand = 0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            attr['demand'] = round(od_data[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time 
            temp_demand += attr['demand']
            attr['price'] = distance_data[origin,destination,int(ts*time_granularity)]*0.91 + duration_data[origin,destination,int(ts*time_granularity)]*time_granularity * 0.39 + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)
print(temp_demand)
# should be able to change 24 to 1, we don't change the number of vehicles across the day
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['additionalVehiclesPeakDemand'] = peak_demand_additional_vehicles
new_data['peakHours'] = peak_hours
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
print(episode_length)
with open('NY_5.json', 'w') as f:
    json.dump(new_data, f)

232072
16


In [4]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
temp_demand = 0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            attr['demand'] = round(0.95*od_data[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time 
            temp_demand += attr['demand']
            attr['price'] = distance_data[origin,destination,int(ts*time_granularity)]*0.91 + duration_data[origin,destination,int(ts*time_granularity)]*time_granularity * 0.39 + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)

# should be able to change 24 to 1, we don't change the number of vehicles across the day
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['additionalVehiclesPeakDemand'] = peak_demand_additional_vehicles
new_data['peakHours'] = [1, 2, 3, 4, 5]
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
print("episode_length", episode_length)
print("fleet_size", fleet_size)
with open('NY_5_test.json', 'w') as f:
    json.dump(new_data, f)

episode_length 16
fleet_size 20524.415999999997
